In [1]:
import rospy
import math
import sys
import select
import actionlib
from nav_msgs.msg import Odometry
from std_srvs.srv import *
from geometry_msgs.msg import Point, Pose, Twist
from ros_simulation.msg import Pos
from sensor_msgs.msg import LaserScan
from ros_simulation.msg import PlanningAction, PlanningGoal
from ros_simulation.srv import Goals

%matplotlib widget 
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import FloatText, VBox, widgets, Layout

position = None
linear_velocity = None

x_plot = []
y_plot = []

x_goal = None
y_goal = None

# Initialize lists to store target positions
targets_set = 0
targets_cancelled = 0

# Create a publisher to publish position information
pub = rospy.Publisher('/pos', Pos, queue_size=1)

# Create an action client for the reaching_goal
action_client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)

# Create a goal message
goal = PlanningGoal()

# Create a publisher to send the target
goal_pub = rospy.Publisher('/goal_topic', Point, queue_size=1)

In [2]:
obstacle_distance = FloatText(description='Obstacle Distance:')

def callback_scan(data):
        obstacle_distance.value = round(min(data.ranges), 2)

In [3]:
# Define callback function for target set button
def target_set_callback(sender):
    global x_goal, y_goal
    # Get x and y coordinates from text boxes
    x_goal = float(x_textbox.value)
    y_goal = float(y_textbox.value)
    
    # To update the goal position
    update_goal(x_goal, y_goal)
    
    goal.target_pose.pose.position.x = x_goal
    goal.target_pose.pose.position.y = y_goal
    
    goal_msg = Point()
    goal_msg.x = goal.target_pose.pose.position.x
    goal_msg.y = goal.target_pose.pose.position.y
    goal_pub.publish(goal_msg)   

    action_client.send_goal(goal)

In [4]:
# Define callback function for target cancel button
def target_cancel_callback(sender):
    action_client.cancel_goal()

In [5]:
goal_reached = 0
goal_cancelled = 0
def get_goals_client():
    # Create a ROS service client for the 'goals_srv' service
    rospy.wait_for_service('goals_srv')
    try:
        # Create a proxy to call the 'goals_srv' service
        get_goals = rospy.ServiceProxy('goals_srv', Goals)
        # Make a request to the service
        response = get_goals()
        # Access the values of reached and cancelled goals from the response -> .VALUE
        goal_reached = response.reached
        goal_cancelled = response.cancelled
        plot_goals(goal_reached, goal_cancelled)
    except rospy.ServiceException as e:
        goal_reached.value = goal_reached.value
        goal_cancelled.value = goal_cancelled.value

In [6]:
def plot(x_plot, y_plot):
    np_x_plot = np.array(x_plot)
    np_y_plot = np.array(y_plot)
    ax[0].plot(np_x_plot, np_y_plot)
    
def plot_goals(goal_reached, goal_cancelled):
    counts = [goal_reached, goal_cancelled]
    colors = ['green', 'red']  # Define colors for each category
    ax[1].bar(goals, counts, color=colors, width=0.6) 
    
# Function to update the goal position
def update_goal(x_goal, y_goal):
    # Remove the previous goal plot line if it exists
    if x_goal is not None and y_goal is not None:
        for line in ax[0].lines:
            if line.get_label() == 'Goal':
                line.remove()

    # Plot the new goal position if both x_goal and y_goal are set
    if x_goal is not None and y_goal is not None:
        ax[0].plot(x_goal, y_goal, 'gx', label='Goal')
        ax[0].legend(loc='upper right')
        fig.canvas.draw()    

# To plot the robot position
np_x_value = FloatText(description='X POSITION:')
np_y_value = FloatText(description='Y POSITION:')

# Create a figure with two subplots
fig, ax = plt.subplots(2, 1, figsize=(8, 8))

# Plot the robot motion
ax[0].set_xlim(-10, 10)
ax[0].set_ylim(-10, 10)
ax[0].set_title("ROBOT MOTION")
ax[0].set_xlabel("x")
ax[0].set_ylabel("y")
np_x_plot = np.array(x_plot)
np_y_plot = np.array(y_plot)
ax[0].plot(np_x_plot,np_y_plot)

# Plot the goal status as a bar plot
goals = ['goal_reached', 'goal_cancelled']
counts = [goal_reached, goal_cancelled]
colors = ['green', 'red']  # Define colors for each category
ax[1].set_ylim(0,16)
ax[1].bar(goals, counts, color=colors, width=0.6)
   
ax[1].set_ylabel('Count')
ax[1].set_title('GOAL STATUS')

# Enable grid lines
ax[1].grid(axis='y', linestyle='--', alpha=0.5)
# Adjust font size and style
ax[1].tick_params(axis='both', labelsize=12)

# Adjust the layout of the subplots
plt.tight_layout()

def odom_callback(data):
    # Store the position and linear velocity information from the odometry message
    position = data.pose.pose.position
    linear_velocity = data.twist.twist.linear

    # Add the robot's position to the plot
    x_plot.append(position.x)
    y_plot.append(position.y)

    # Update the FloatText widgets with the latest position values
    np_x_value.value = x_plot[-1]
    np_y_value.value = y_plot[-1]

    msg = Pos()
    msg.x = position.x
    msg.y = position.y
    msg.vx = linear_velocity.x
    msg.vy = linear_velocity.y

     # Publish the message
    pub.publish(msg)

    # Call the function to retrieve the goals information
    get_goals_client()

    # Update the plot with the robot's position and target position
    plot(x_plot, y_plot)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# initialize the node
rospy.init_node("Client_jupyter")

# Create a subscriber to listen for odometry information
sub_odom = rospy.Subscriber('/odom', Odometry, odom_callback)

# Create a subscriber to listen for laser information
sub_scan = rospy.Subscriber('/scan', LaserScan, callback_scan)

# Call the function to retrive the goals information
get_goals_client()

# Create text boxes for target coordinates
x_textbox = widgets.FloatText(description='X:', layout=Layout(width='150px'))
y_textbox = widgets.FloatText(description='Y:', layout=Layout(width='150px'))

# Create buttons to set and cancel targets
target_set_button = widgets.Button(description='Set Target')
target_cancel_button = widgets.Button(description='Cancel Target')

# Attach callback functions to buttons
target_set_button.on_click(target_set_callback)
target_cancel_button.on_click(target_cancel_callback)

# Display widgets
display(VBox([np_x_value,np_y_value]))
display(widgets.HBox([x_textbox, y_textbox, target_set_button, target_cancel_button]))
display(VBox([obstacle_distance]))